# Content with notebooks

You can also create content with Jupyter Notebooks. This means that you can include
code blocks and their outputs in your book.

## Markdown + notebooks

As it is markdown, you can embed images, HTML, etc into your posts!

![](https://myst-parser.readthedocs.io/en/latest/_static/logo-wide.svg)

You can also $add_{math}$ and

$$
math^{blocks}
$$

or

$$
\begin{aligned}
\mbox{mean} la_{tex} \\ \\
math blocks
\end{aligned}
$$

But make sure you \$Escape \$your \$dollar signs \$you want to keep!

## MyST markdown

MyST markdown works in Jupyter Notebooks as well. For more information about MyST markdown, check
out [the MyST guide in Jupyter Book](https://jupyterbook.org/content/myst.html),
or see [the MyST markdown documentation](https://myst-parser.readthedocs.io/en/latest/).

## Code blocks and outputs

Jupyter Book will also embed your code blocks and output in your book.
For example, here's some sample Matplotlib code:

In [1]:
%pip install requests bs4 pandas

from bs4 import BeautifulSoup
import requests
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [2]:
url = "https://www.uc3m.es/bachelor-degree/data-science"
response = requests.get(url).text

soup = BeautifulSoup(response, 'html.parser')

soup_quality = soup.find(id="quality")
print(soup_quality)


<div class="marcoParrafo" id="quality">
<h2>Quality</h2>
</div>


In [52]:
import json


def scrape_airbnb(check_in, check_out):
    url = f"https://www.airbnb.com/s/Madrid/homes?place_id=ChIJgTwKgJcpQg0RaSKMYcHeNsQ&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=october&flexible_trip_dates%5B%5D=september&date_picker_type=calendar&search_type=filter_change&tab_id=home_tab&query=Madrid&monthly_start_date=2023-12-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=1&channel=EXPLORE&flexible_trip_lengths%5B%5D=weekend_trip&source=structured_search_input_header&checkin={check_in}&checkout={check_out}&adults=1"
    response = requests.get(url).text

    soup = BeautifulSoup(response, "html.parser")

    script_tag = soup.find("script", {"id": "data-state", "type": "application/json"})
    if script_tag:
        json_data = json.loads(script_tag.string)
        return json_data
    else:
        return "Data not found"


def extract_data(json_data, check_in, check_out):
    result = []
    length = len(
        json_data["niobeMinimalClientData"][1][1]["data"]["presentation"][
            "staysSearch"
        ]["results"]["searchResults"]
    )

    for i in range(length - 1):
        # LOCATION
        location = json_data["niobeMinimalClientData"][1][1]["data"]["presentation"][
            "staysSearch"
        ]["results"]["searchResults"][i]["listing"]["coordinate"]
        latitude = location["latitude"]
        longitude = location["longitude"]

        # PRICE
        price = json_data["niobeMinimalClientData"][1][1]["data"]["presentation"][
            "staysSearch"
        ]["results"]["searchResults"][i]["pricingQuote"]["rate"]["amount"]

        room_type = json_data["niobeMinimalClientData"][1][1]["data"]["presentation"][
            "staysSearch"
        ]["results"]["searchResults"][i]["listing"]["roomTypeCategory"]

        result.append(
            {
                "check_in": check_in,
                "check_out": check_out,
                "latitude": latitude,
                "longitude": longitude,
                "price": price,
                "room_type": room_type,
            }
        )

    return result


airbnb_raw_data = []
year = 2024
for month in range(1, 12):
    for day in range(1, 28):  # make it easy for february
        check_in = f"{year}-{month}-{day}"
        check_out = f"{year}-{month}-{day + 1}"
        json_data = scrape_airbnb(check_in, check_out)
        airbnb_raw_data.extend(extract_data(json_data, check_in, check_out))

# Flatten the list of lists into a single list of dictionaries
flat_data = [item for sublist in airbnb_raw_data for item in sublist]

airbnb_data = pd.DataFrame(
    airbnb_raw_data,
    columns=["check_in", "check_out", "latitude", "longitude", "price", "room_type"],
)
airbnb_data["check_in"] = pd.to_datetime(airbnb_data["check_in"])
airbnb_data["check_out"] = pd.to_datetime(airbnb_data["check_out"])

print(airbnb_data.head())
airbnb_data.to_csv("airbnb_data.csv", index=False)

    check_in  check_out   latitude  longitude  price     room_type
0 2024-01-01 2024-01-02  40.434800  -3.683460     47  private_room
1 2024-01-01 2024-01-02  40.406610  -3.674030     27  private_room
2 2024-01-01 2024-01-02  40.426480  -3.704340     56  private_room
3 2024-01-01 2024-01-02  40.407969  -3.701829     85   entire_home
4 2024-01-01 2024-01-02  40.410685  -3.703990     33  private_room


In [12]:
# Fixing random state for reproducibility
np.random.seed(19680801)

N = 10
data = [np.logspace(0, 1, 100) + np.random.randn(100) + ii for ii in range(N)]
data = np.array(data).T
cmap = plt.cm.coolwarm
rcParams['axes.prop_cycle'] = cycler(color=cmap(np.linspace(0, 1, N)))


from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color=cmap(0.), lw=4),
                Line2D([0], [0], color=cmap(.5), lw=4),
                Line2D([0], [0], color=cmap(1.), lw=4)]

fig, ax = plt.subplots(figsize=(10, 5))
lines = ax.plot(data)
ax.legend(custom_lines, ['Cold', 'Medium', 'Hot']);

NameError: name 'np' is not defined

There is a lot more that you can do with outputs (such as including interactive outputs)
with your book. For more information about this, see [the Jupyter Book documentation](https://jupyterbook.org)